In [68]:
# Firstly, we must import some libs
import os
import numpy as np
import pandas as pd
import json
import yaml
import torch

from sklearn.model_selection import train_test_split
import shutil as sh
from collections import Counter

In [69]:
p_dir = '/home/zyp/w/d2l/detect_outfits'

In [75]:
conf_thres_val = 0.05
epochs = 60

In [70]:
json_file_path = p_dir + '/data/origin/train.json'
data = json.load(open(json_file_path, 'r'))

In [71]:
cats = pd.DataFrame(data["categories"])[['id', 'name']].set_index('id')
cats_dict = cats.squeeze().to_dict()
cats_dict

{87: 'belt', 1034: 'sunglasses', 131: 'boot', 318: 'cowboy_hat', 588: 'jacket'}

In [72]:
def cc2yolo_bbox(img_width, img_height, bbox):
    dw = 1. / img_width
    dh = 1. / img_height
    x = bbox[0] + bbox[2] / 2.0
    y = bbox[1] + bbox[3] / 2.0
    w = bbox[2]
    h = bbox[3]
 
    x = x * dw
    w = w * dw
    y = y * dh
    h = h * dh
    return (x, y, w, h)

def yolo2cc_bbox(img_width, img_height, bbox):
    x = (bbox[0] - bbox[2] * 0.5) * img_width
    y = (bbox[1] - bbox[3] * 0.5) * img_height
    w = bbox[2] * img_width
    h = bbox[3] * img_height
    
    return (x, y, w, h)

def cat_to_id():
    cat_id_map = {}
    num = 0
    for cate in data['categories']:
        cat_id_map[cate['id']] = num
        num+=1
    return cat_id_map

cat_id_map = cat_to_id()
print(cat_id_map)

re_cat_id_map = dict(zip(cat_id_map.values(), cat_id_map.keys()))
print(re_cat_id_map)

{87: 0, 1034: 1, 131: 2, 318: 3, 588: 4}
{0: 87, 1: 1034, 2: 131, 3: 318, 4: 588}


In [73]:
imgs = pd.DataFrame(data['images'])[["id", "file_name", "width", "height"]].set_index('id')

small_set = 0
if small_set:
    imgs = imgs[:10]
imgs

,file_name,width,height
id,,,
9860841628484337660,88d8bf3754317ffc.jpg,1024,681
15984033263460081658,ddd2b190ea90dffa.jpg,768,1024
76077631043502082,010e4833cdb38002.jpg,1024,683
18065680256228130812,fab6307a1a43fffc.jpg,683,1024
9491379842992996352,83b827ae01e68000.jpg,1024,768
...,...,...,...
10412147159402939302,907f60acf14e03a6.jpg,1024,819
4866906309960860041,438ab980c0530189.jpg,1024,681
10441816352675791240,90e8c8a64f1b0188.jpg,1024,1024


In [74]:
train_imgs, valid_imgs = train_test_split(imgs, test_size=0.2, random_state=233)
print(len(train_imgs), len(valid_imgs))
valid_imgs

2449 613


,file_name,width,height
id,,,
2857187227633385663,27a6c4a53fdf00bf.jpg,683,1024
1855409563895400077,19bfbd2b00d9068d.jpg,679,1024
2581269785293458045,23d283318ddd8a7d.jpg,683,1024
9539337138986144032,8462889619f5d120.jpg,805,1024
15697918461187198072,d9da35ab581c0c78.jpg,683,1024
...,...,...,...
17878807805699515256,f81e48f92a336378.jpg,1024,682
6511218266802815244,5a5c7e8407d1010c.jpg,1024,747
16454855895458090054,e45b64415f058c46.jpg,681,1024


In [76]:
!python ~/w/github/yolov5/detect.py --weight ~/w/d2l/detect_outfits/log/bln_epochs{epochs}/weights/last.pt \
                               --source ~/w/d2l/detect_outfits/data/yolo2/valid/images \
                               --name ~/w/d2l/detect_outfits/log/bln_epochs{epochs}/{conf_thres_val}-exp \
                               --save-txt --save-conf --iou-thres 0.5 --conf-thres {conf_thres_val} --img 640 

detect: weights=['/home/zyp/w/d2l/detect_outfits/log/bln_epochs60/weights/last.pt'], source=/home/zyp/w/d2l/detect_outfits/data/yolo2/valid/images, data=../../github/yolov5/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.05, iou_thres=0.5, max_det=1000, device=, view_img=False, save_txt=True, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=../../github/yolov5/runs/detect, name=/home/zyp/w/d2l/detect_outfits/log/bln_epochs60/0.05-exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.1-237-g50b886d Python-3.8.12 torch-1.11.0+cu102 CPU

Fusing layers... 
Model summary: 213 layers, 7023610 parameters, 0 gradients, 15.8 GFLOPs
image 1/613 /home/zyp/w/d2l/detect_outfits/data/yolo2/valid/images/0003bc2dabab095a.jpg: 448x640 1 sunglasses, Done. (0.062s)
image 2/613 /home/zyp/w/d2l/detect_outfits/data/yolo2/valid/images/000464f9fff223ab.jpg: 640x544 1 c

image 59/613 /home/zyp/w/d2l/detect_outfits/data/yolo2/valid/images/158fecad158b049c.jpg: 640x448 1 jacket, Done. (0.051s)
image 60/613 /home/zyp/w/d2l/detect_outfits/data/yolo2/valid/images/15d0bfbcc5df04c6.jpg: 640x480 2 sunglassess, Done. (0.056s)
image 61/613 /home/zyp/w/d2l/detect_outfits/data/yolo2/valid/images/169134c7b29d8556.jpg: 480x640 2 sunglassess, 5 jackets, Done. (0.056s)
image 62/613 /home/zyp/w/d2l/detect_outfits/data/yolo2/valid/images/171e637cba328188.jpg: 480x640 1 sunglasses, Done. (0.053s)
image 63/613 /home/zyp/w/d2l/detect_outfits/data/yolo2/valid/images/18546c5126c38a90.jpg: 448x640 3 jackets, Done. (0.051s)
image 64/613 /home/zyp/w/d2l/detect_outfits/data/yolo2/valid/images/1886ca4b839280f8.jpg: 448x640 5 sunglassess, 4 cowboy_hats, Done. (0.071s)
image 65/613 /home/zyp/w/d2l/detect_outfits/data/yolo2/valid/images/18b7a974c1c08a9c.jpg: 480x640 2 sunglassess, Done. (0.054s)
image 66/613 /home/zyp/w/d2l/detect_outfits/data/yolo2/valid/images/1921dcd0887580f3.jpg

image 123/613 /home/zyp/w/d2l/detect_outfits/data/yolo2/valid/images/2c368797264b046a.jpg: 640x640 1 jacket, Done. (0.086s)
image 124/613 /home/zyp/w/d2l/detect_outfits/data/yolo2/valid/images/2c633ed2bb6604c9.jpg: 448x640 3 sunglassess, Done. (0.051s)
image 125/613 /home/zyp/w/d2l/detect_outfits/data/yolo2/valid/images/2da37921f366418c.jpg: 448x640 1 sunglasses, 2 cowboy_hats, Done. (0.068s)
image 126/613 /home/zyp/w/d2l/detect_outfits/data/yolo2/valid/images/2e3f67f6c6f803e8.jpg: 448x640 1 sunglasses, Done. (0.054s)
image 127/613 /home/zyp/w/d2l/detect_outfits/data/yolo2/valid/images/2ec488947bad8565.jpg: 416x640 4 jackets, Done. (0.049s)
image 128/613 /home/zyp/w/d2l/detect_outfits/data/yolo2/valid/images/2f733a0323b2006a.jpg: 640x512 1 sunglasses, 1 jacket, Done. (0.062s)
image 129/613 /home/zyp/w/d2l/detect_outfits/data/yolo2/valid/images/2f9a9438396a081c.jpg: 448x640 1 jacket, Done. (0.053s)
image 130/613 /home/zyp/w/d2l/detect_outfits/data/yolo2/valid/images/2fab48dc1d0b03c6.jpg

image 186/613 /home/zyp/w/d2l/detect_outfits/data/yolo2/valid/images/453d43e569380568.jpg: 640x640 3 sunglassess, Done. (0.078s)
image 187/613 /home/zyp/w/d2l/detect_outfits/data/yolo2/valid/images/45b0d6ec5e2020e7.jpg: 640x448 1 jacket, Done. (0.056s)
image 188/613 /home/zyp/w/d2l/detect_outfits/data/yolo2/valid/images/463ced6428b98434.jpg: 640x448 1 cowboy_hat, 2 jackets, Done. (0.053s)
image 189/613 /home/zyp/w/d2l/detect_outfits/data/yolo2/valid/images/473dbdd7162089c8.jpg: 480x640 2 sunglassess, Done. (0.059s)
image 190/613 /home/zyp/w/d2l/detect_outfits/data/yolo2/valid/images/47ed77aecd3183a0.jpg: 480x640 1 sunglasses, 1 boot, 3 jackets, Done. (0.058s)
image 191/613 /home/zyp/w/d2l/detect_outfits/data/yolo2/valid/images/489e307560168805.jpg: 480x640 1 sunglasses, Done. (0.059s)
image 192/613 /home/zyp/w/d2l/detect_outfits/data/yolo2/valid/images/489fb2c6dcbd8bcf.jpg: 640x480 8 sunglassess, Done. (0.060s)
image 193/613 /home/zyp/w/d2l/detect_outfits/data/yolo2/valid/images/49577a

image 250/613 /home/zyp/w/d2l/detect_outfits/data/yolo2/valid/images/6057028d487b0904.jpg: 640x480 1 sunglasses, Done. (0.059s)
image 251/613 /home/zyp/w/d2l/detect_outfits/data/yolo2/valid/images/6159e5226829004e.jpg: 448x640 2 sunglassess, 2 jackets, Done. (0.052s)
image 252/613 /home/zyp/w/d2l/detect_outfits/data/yolo2/valid/images/61710355af4a03be.jpg: 640x640 1 sunglasses, 1 jacket, Done. (0.077s)
image 253/613 /home/zyp/w/d2l/detect_outfits/data/yolo2/valid/images/6257069d72c00af4.jpg: 640x640 1 sunglasses, 3 jackets, Done. (0.076s)
image 254/613 /home/zyp/w/d2l/detect_outfits/data/yolo2/valid/images/62d4cdd03699855c.jpg: 448x640 9 sunglassess, Done. (0.065s)
image 255/613 /home/zyp/w/d2l/detect_outfits/data/yolo2/valid/images/62de6a86f0b50cdb.jpg: 448x640 1 sunglasses, Done. (0.062s)
image 256/613 /home/zyp/w/d2l/detect_outfits/data/yolo2/valid/images/63b50ab1ce0f055e.jpg: 448x640 2 sunglassess, Done. (0.062s)
image 257/613 /home/zyp/w/d2l/detect_outfits/data/yolo2/valid/images/

image 314/613 /home/zyp/w/d2l/detect_outfits/data/yolo2/valid/images/7dbe1dbf556007c1.jpg: 448x640 1 jacket, Done. (0.053s)
image 315/613 /home/zyp/w/d2l/detect_outfits/data/yolo2/valid/images/7ed369a9a534883a.jpg: 640x544 1 sunglasses, Done. (0.068s)
image 316/613 /home/zyp/w/d2l/detect_outfits/data/yolo2/valid/images/7ed3aaf907c3812e.jpg: 448x640 1 sunglasses, Done. (0.054s)
image 317/613 /home/zyp/w/d2l/detect_outfits/data/yolo2/valid/images/7f02e9a1ef000897.jpg: 448x640 4 sunglassess, Done. (0.053s)
image 318/613 /home/zyp/w/d2l/detect_outfits/data/yolo2/valid/images/7febea95b0948cad.jpg: 480x640 1 sunglasses, Done. (0.056s)
image 319/613 /home/zyp/w/d2l/detect_outfits/data/yolo2/valid/images/803eaca98c5505c6.jpg: 480x640 2 sunglassess, 2 cowboy_hats, Done. (0.056s)
image 320/613 /home/zyp/w/d2l/detect_outfits/data/yolo2/valid/images/806ffeef523e8694.jpg: 448x640 Done. (0.053s)
image 321/613 /home/zyp/w/d2l/detect_outfits/data/yolo2/valid/images/80c0fd83f0430989.jpg: 640x448 1 sung

image 377/613 /home/zyp/w/d2l/detect_outfits/data/yolo2/valid/images/93feeca33e4c8603.jpg: 640x480 1 sunglasses, Done. (0.061s)
image 378/613 /home/zyp/w/d2l/detect_outfits/data/yolo2/valid/images/942ba48015dd0497.jpg: 480x640 6 sunglassess, 1 jacket, Done. (0.056s)
image 379/613 /home/zyp/w/d2l/detect_outfits/data/yolo2/valid/images/948c254ecd2c0031.jpg: 448x640 1 jacket, Done. (0.054s)
image 380/613 /home/zyp/w/d2l/detect_outfits/data/yolo2/valid/images/948cef9a588986f6.jpg: 480x640 2 sunglassess, Done. (0.057s)
image 381/613 /home/zyp/w/d2l/detect_outfits/data/yolo2/valid/images/94f49f03904f8326.jpg: 480x640 3 sunglassess, Done. (0.056s)
image 382/613 /home/zyp/w/d2l/detect_outfits/data/yolo2/valid/images/950fe4f91cd651b5.jpg: 448x640 2 boots, Done. (0.052s)
image 383/613 /home/zyp/w/d2l/detect_outfits/data/yolo2/valid/images/952f2b57105b84ea.jpg: 640x448 1 sunglasses, Done. (0.054s)
image 384/613 /home/zyp/w/d2l/detect_outfits/data/yolo2/valid/images/9535cb8ec239e278.jpg: 640x448 3

image 441/613 /home/zyp/w/d2l/detect_outfits/data/yolo2/valid/images/b5d3508b436a05df.jpg: 640x448 4 sunglassess, 2 boots, Done. (0.053s)
image 442/613 /home/zyp/w/d2l/detect_outfits/data/yolo2/valid/images/b5e6852e110e013d.jpg: 480x640 1 sunglasses, 1 cowboy_hat, Done. (0.059s)
image 443/613 /home/zyp/w/d2l/detect_outfits/data/yolo2/valid/images/b65ff36802f1841c.jpg: 480x640 1 jacket, Done. (0.055s)
image 444/613 /home/zyp/w/d2l/detect_outfits/data/yolo2/valid/images/b6694a942be350aa.jpg: 640x640 3 sunglassess, 1 cowboy_hat, Done. (0.079s)
image 445/613 /home/zyp/w/d2l/detect_outfits/data/yolo2/valid/images/b6d73b22171c8675.jpg: 640x320 1 sunglasses, 1 cowboy_hat, 1 jacket, Done. (0.056s)
image 446/613 /home/zyp/w/d2l/detect_outfits/data/yolo2/valid/images/b733f8a22157b128.jpg: 480x640 Done. (0.059s)
image 447/613 /home/zyp/w/d2l/detect_outfits/data/yolo2/valid/images/b742bed9dacf8160.jpg: 640x480 1 jacket, Done. (0.057s)
image 448/613 /home/zyp/w/d2l/detect_outfits/data/yolo2/valid/i

image 504/613 /home/zyp/w/d2l/detect_outfits/data/yolo2/valid/images/cdabe49719bd803f.jpg: 384x640 1 sunglasses, Done. (0.047s)
image 505/613 /home/zyp/w/d2l/detect_outfits/data/yolo2/valid/images/cdc33c31888801f5.jpg: 640x448 2 sunglassess, 1 jacket, Done. (0.054s)
image 506/613 /home/zyp/w/d2l/detect_outfits/data/yolo2/valid/images/ce02bb70a7bf0a12.jpg: 448x640 Done. (0.052s)
image 507/613 /home/zyp/w/d2l/detect_outfits/data/yolo2/valid/images/ce044dbab7bc0153.jpg: 480x640 1 sunglasses, Done. (0.056s)
image 508/613 /home/zyp/w/d2l/detect_outfits/data/yolo2/valid/images/ce082490338104e9.jpg: 640x640 1 sunglasses, Done. (0.074s)
image 509/613 /home/zyp/w/d2l/detect_outfits/data/yolo2/valid/images/ce328326fef61060.jpg: 448x640 1 boot, Done. (0.051s)
image 510/613 /home/zyp/w/d2l/detect_outfits/data/yolo2/valid/images/cf80f81736658149.jpg: 448x640 10 jackets, Done. (0.053s)
image 511/613 /home/zyp/w/d2l/detect_outfits/data/yolo2/valid/images/cfb95ad5ce878737.jpg: 448x640 2 sunglassess, D

image 568/613 /home/zyp/w/d2l/detect_outfits/data/yolo2/valid/images/eb3b45182b920a0a.jpg: 640x448 1 jacket, Done. (0.052s)
image 569/613 /home/zyp/w/d2l/detect_outfits/data/yolo2/valid/images/eb6f667c573c81f8.jpg: 448x640 6 jackets, Done. (0.054s)
image 570/613 /home/zyp/w/d2l/detect_outfits/data/yolo2/valid/images/ebc9e35899a802d1.jpg: 448x640 1 sunglasses, 1 jacket, Done. (0.053s)
image 571/613 /home/zyp/w/d2l/detect_outfits/data/yolo2/valid/images/ec72ec56740eb3a4.jpg: 640x448 1 cowboy_hat, Done. (0.053s)
image 572/613 /home/zyp/w/d2l/detect_outfits/data/yolo2/valid/images/ecd04fd513e10b10.jpg: 448x640 4 jackets, Done. (0.056s)
image 573/613 /home/zyp/w/d2l/detect_outfits/data/yolo2/valid/images/edf3ac4d22ae0123.jpg: 448x640 Done. (0.052s)
image 574/613 /home/zyp/w/d2l/detect_outfits/data/yolo2/valid/images/ef8715a84afa8be7.jpg: 640x384 1 jacket, Done. (0.049s)
image 575/613 /home/zyp/w/d2l/detect_outfits/data/yolo2/valid/images/efe69f1ca4fcc0a8.jpg: 640x448 1 boot, 1 jacket, Done.

In [77]:
exp_dir = p_dir + f'/log/bln_epochs{epochs}/{conf_thres_val}-exp'
os.makedirs(exp_dir, exist_ok=True)
print(exp_dir)

no_detect = []

def yolo_to_cc():
    results = []
    for img in valid_imgs.reset_index().to_numpy():
        img_id = img[0]
        img_name = img[1].split('.')[0]
        img_w = img[2]
        img_h = img[3]
        
        ann_file = f'{exp_dir}/labels/{img_name}.txt'

        if not os.path.exists(ann_file):
            no_detect.append([img_id, f'{img_name}.jpg'])
            continue
        with open(ann_file) as f:
            lines = f.readlines()
            for line in lines:
                
                predicts = line.strip('\n').split(' ')
                cat_id = int(predicts[0])
                bbox = list(map(float, predicts[1:-1]))
                score = float(predicts[-1])
                bbox = yolo2cc_bbox(img_w, img_h, bbox)
                pred = {'image_id': img_id,
                        'category_id': re_cat_id_map[cat_id],
                        'bbox': bbox,
                        'score': score}
                results.append(pred)
    return results

cc_ann_pre = yolo_to_cc()
print(len(cc_ann_pre))

/home/zyp/w/d2l/detect_outfits/log/bln_epochs60/0.05-exp
1593


In [78]:
answer_file = exp_dir +'/answer.json'
with open(answer_file, 'w') as f:
    json.dump(cc_ann_pre, f)
print(answer_file)

/home/zyp/w/d2l/detect_outfits/log/bln_epochs60/0.05-exp/answer.json


In [79]:
len(no_detect)

22